In [ ]:
#!pip install urllib3 --user
#!pip install requests_html --user
#!pip install cffi --user --version 1.12.3

In [4]:
import requests
import bs4
from bs4 import BeautifulSoup
from bs4 import Comment
import pandas as pd
import requests_html
from requests_html import AsyncHTMLSession


### employees for Ravensburg, Mannheim, Heidenheim, Karlsruhe

In [5]:
locations = ["ravensburg", "mannheim", "heidenheim", "karlsruhe"]
job_title_match = ["akademisch", "professor", "studiengangsleiter", "wisschenschaftl"]

# get current year to save in applicable folder
current_year = pd.datetime.now().year

<ipython-input-5-b33a2edc3323>:5: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  current_year = pd.datetime.now().year


In [7]:
for location in locations:
    url = "https://www." + location + ".dhbw.de/dhbw-" + location + "/ansprechpersonen"
    session = AsyncHTMLSession()
    r = await session.get(url)
    await r.html.arender()
    page_soup = BeautifulSoup(r.html.html, "html.parser")
    people = page_soup.find_all('a', class_="accordion-toggle collapsed")

    employees = pd.DataFrame(data=None, columns=["employee", "first_matched_job_title"])

    for person in people:
        splitted_person = person.get_text().strip().split("\n")

        for job_title in splitted_person[2:]:
            job_title_formatted = job_title.strip().lower()
            if job_title_formatted != None and any(x in job_title_formatted for x in job_title_match):
                new_employee = {"employee" : splitted_person[0], "first_matched_job_title" : job_title.strip()}
                employees = employees.append(new_employee, ignore_index=True)
                break

    employees.head()
    employees.to_csv(f'../data/{current_year}/employees_{location}.csv', index=False)

### test purposes

In [39]:
test_url = "https://www.mannheim.dhbw.de/dhbw-mannheim/ansprechpersonen"
session = AsyncHTMLSession()
r = await session.get(url)
await r.html.arender()
page_soup = BeautifulSoup(r.html.html, "html.parser")
results = page_soup.find_all('a', class_="accordion-toggle collapsed")

job_title_match = ["akademisch", "professor", "studiengangsleiter", "wisschenschaftl"]

employees = pd.DataFrame(columns=["employee", "first_matched_job_title"])

for result in results:
    s = result.get_text().strip().split("\n")
    print(s)
    for item in s[2:]:
        item_formatted = item.strip().lower()
        if item != None and any(x in item_formatted for x in job_title_match):
            employees = employees.append({"employee" : s[0], "first_matched_job_title" : item.strip()}, ignore_index=True)
            break

employees.head()

['Ahrens,  Britta', '                                    ', '                                        Sekretariat Studiengang Elektrotechnik']
['Ahrens,  Ulrike', '                                    ', '                                        Sekretariat Studiengang Maschinenbau']
['Arheidt,  Patrick', '                                    ', '                                        Gebäudemanagement, Beschaffung, Sicherheit und Technik']
['Avella, Prof. Dr. Felice-Alfredo', '                                    ', '                                        Professor Fakultät Wirtschaft']
['Bähr,  Sabine', '                                    ', '                                        Sekretariat Fakultät WirtschaftSekretariat Studiengang RSW-Steuern und Prüfungswesen']
['Barth,  Heike', '                                    ', '                                        Sekretariat Studiengang Wirtschaftsingenieurwesen']
['Bartl,  Bruno', '                                    ', '            

,employee,first_matched_job_title
0,"Avella, Prof. Dr. Felice-Alfredo",Professor Fakultät Wirtschaft
1,"Berkling, Ph.D., Prof. Kay Margarethe",Professorin Fakultät Technik
2,"Bonaudo, Patricia",akademische Mitarbeiterin im Bereich „Lerntech...
3,"Brandstetter, Christian",akademischer Mitarbeiter im Bereich Entreprene...
4,"Brunner, Marina",akademische Mitarbeiterin im Bereich Lerntechn...
